### LangChain 기초

1. 체인(chain)에 대한 이해: 기본 LLM 체인(Prompt + LLM) | 멀티 체인
2. 프롬프트 만들기: Prompt Template 이해 및 적용
3. LLM 모델 구조: LLM 클래스와 ChatModel 모델 클래스 구분
4. LLM 모델 튜닝: 모델 파라미터 설정
5. RAG(Retrieval-Augmented Generation) 기법 이해: 웹 문서에 대한 QA 챗봇 만들기

기본 체인(Prompt + LLM)

기본 체인은 LLM 기반 애플리케이션 개발에서 핵심적인 개념 중 하나입니다. 이러한 체인은 사용자의 입력(프롬프트)을 받아 LLM을 통해 적절한 응답이나 생성하는 구조를 말함

기본 체인 구성 요소
1. 프롬프트: 사용자와 언어 모델 간의 대화에서 질문이나 요청의 형태로 제시되는 입력문
2. LLM: GPT, GEMINI 등의 생성형 언어모델델

일반적인 작동 방식
1. 프롬프트 생성
2. LLM 처리
3. 응답 변환

In [3]:
# OpenAI 인증키 설정

import os

# gpt api key 입력력
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'

1) Prompt + LLM
- 가장 기본적이고 일반적인 사용 사례
- 프로프트 템플릿과 모델을 연결

In [5]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model = "gpt-3.5-turbo-0125")

#chain 설정
result = llm.invoke("지구의 자전 주기는?")

In [8]:
result.content

'지구의 자전 주기는 약 24시간 입니다. 따라서 하루는 24시간으로 나뉘어 있으며, 지구가 한 바퀴를 자전하는데 약 24시간이 소요됩니다.'

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Prompt 템플릿 설정(시스템 메시지를 입력하여 LLM의 기본적인 역할을 부여여)
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: (input)")
# GPT 버전 설정
llm = ChatOpenAI(model= "gpt-3.5-turbo-0125")
# LLM의 출력을 답변의 콘텐츠(문자열열)만 출력하도록 설정
output_parser = StrOutputParser()

# LCEL chining '|를 통해 체인의 구성요소의 순서를 설정 
# promt 추력 -> LLM(GPT)의 답변 -> AI message 출력
chain = prompt | llm | output_parser

chain.invoke({"input": "지구의 자전 주기는?"})

"Sure, I'll do my best to help. What would you like to know about astronomy?"

## 멀티 체인

여러 개의 체인을 연결하거나 복합적으로 작용하는 것은 멀티 체인(Multi-chain) 구조. 이러한 구조는 각기 다른 목적을 갖는 여러 체인을 조합하여, 입력 데이터를 다양한 방식으로 처리하고 최종적인 결과를 도출할 수 있도록 한다.

1. 순차적인 체인 연결
- 2개의 체인(chain1, chain2) 정의
- chain1은 한국어 단어를 영어로 번역하는 작업 수행
- chain 2는 chain1에서 출력한 값(미래)을 입력값으로 받아서, 이 번역된 단어를 english_word 변수에 저장. 다음으로, 이 변수를 사용하여 두번째 체인(chain2)의 입력으로 제공하고, 영어 단어의 뜻을 한국어로 설명하는 작업을 수행

※ 주의해야 할 점: invoke 메소드는 입력으로 딕셔너리 객체를 받기 때문에, 디셔너리의 키는 프롬프트 템플릿에서 정의된 변수명({korean_word})과 일치해야 함

In [16]:
prompt1 = ChatPromptTemplate.from_template("trainslates {korean_word} to English")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
    )

llm = ChatOpenAI(model= 'gpt-3.5-turbo-0125')

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word": "미래"})

'future'

In [17]:
chain2 =(
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
         )

chain2.invoke({"korean_word": "미래"})

'미래는 "지금으로부터 상상하는 시점이 멀리 떨어진 시일"을 의미합니다. 이는 어떤 일이나 사건이 일어날 때 미래에 대해 말할 때 사용하는 용어입니다. 또한 미래는 "어떤 일이나 현상이 앞으로 일어날 것이나 될 것이라고 생각되는 시기"를 의미하기도 합니다. 이는 예측이나 계획에 관련된 맥락에서 사용됩니다.'

## 1-3 프롬프트 템플릿

Prompt Template는 단일 문장 또는 간단한 명령을 입력하여 단일 문장 또는 간단한 응답을 생성하는 데 사용되는 프롬프트를 구성할 수 있는 문자열 템플릿을 의미함

- Prompt Template + LLMs (단일 문장 입력 -> 단일 문장 출력)
- 문자열 프롬프트를 위한 템플릿을 생성. Python의 문자열 포매팅 구문을 사용.
- 내용: 지시사항, 몇 가지 예시, 특정 맥락 및 질문 등

템플릿 사용 이유: 재사용성과 사용의 유연성을 위해 사용

In [19]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의

template_text = "안녕하세요. 제 이름은 {name}이고, 나이는 {age}살 입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name = "홍길동", age = 30)

filled_prompt

'안녕하세요. 제 이름은 홍길동이고, 나이는 30살 입니다.'

In [20]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)

combined_prompt = (
    prompt_template
    + PromptTemplate.from_template('\n\n아버지를 아버지라 부를 수 없습니다.')
    + PromptTemplate.from_template('\n\n{language}로 번역해주세요.')
)

combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], input_types={}, partial_variables={}, template='안녕하세요. 제 이름은 {name}이고, 나이는 {age}살 입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [23]:
combined_prompt.format(name = '홍길도', age = 30, language = '영어')

'안녕하세요. 제 이름은 홍길도이고, 나이는 30살 입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

## 1-3-3 챗 프롬프트 템플릿

ChatPromptTemplate는 대화형 상황에서 여러 메시지를 입력을 기반으로 단일 메시지 응답을 생성하는 데 사용됨.
이는 대화형 모델이나 챗봇 개발에 주로 사용되며, 입력은 여러 메시지를 원소로 갖는 리스트로 구성됨

각 메시지는 역할(role)과 내용(content)으로 구성됨

- ChatPromptTemplate + ChatModels(여러 메시지 입력 -> 단일 메시지)
- 채팅 메시지는 역할(role)과 내용(content)이 짝을 이루는 형태
- 예시 OpenAI는 AI Assistant, Human, System 등의 역할로 구성

1. Message 유형
- SystemMessage: 시스템의 기능을 설명
- HumanMessage: 사용자의 질문을 나타냄
- AIMessage: AI 모델의 응답을 제공
- FunctionMessage: 특정 함수 호출의 결과를 나타냄
- ToolMessage: 도구 호출의 결과를 나타냄

In [26]:
from langchain_core.prompts import ChatPromptTemplate

#
chat_prompt = ChatPromptTemplate([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input = "태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [27]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input":"태양계에서 가장 큰 행성은 무엇인가요?"})

'가장 큰 행성은 목성입니다. 목성은 태양계에서 가장 많은 질량을 가지고 있으며, 79개의 위성을 가지고 있습니다.'

In [29]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_prompt.format_messages(user_input = "태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [33]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input" : "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 139,820km로 태양계에서 가장 큰 크기를 가지고 있습니다.'

## 1-4-1 LangChain 모델 유형


각기 다른 특성과 용도에 따라 LangChain 모델을 활용
1. LLM: 텍스트 문자열을 입력으로 받아 처리한 후, 텍스트 문자열로 반환, 광범위한 언어 이해 및 텍스트 생성 작어에 사용됨
2. Chat Model: 메시지와 리스트를 입력으로 받고, 하나의 메시지를 반환, Chat Model은 대화의 맥학을 유지하면서 적절한 응갑을 생성하는 데 중점을 둔 모델

## 5. Model Parameter

In [ ]:
# LLM

from langchain_openai import OpenAI

llm = OpenAI()

llm.invoke("한국의 대표적인 관광지 3군데를 추천해주세요.")

'\n\n1. 경복궁 - 서울에 위치한 대한민국의 대표적인 궁궐로, 조선 왕조의 본궁이었던 곳입니다. 아름다운 전통 건축물과 아름다운 정원을 감상할 수 있으며, 궁궐 내부에는 국립민속박물관과 국립고궁박물관 등 역사 및 문화를 배울 수 있는 곳도 있습니다.\n\n2. 제주도 - 한국의 남쪽 끝에 위치한 아름다운 섬으로, 자연과 문화를 함께 즐길 수 있는 곳입니다. 제주도는 세계자연유산으로 등재된 한라산의 아름다운 풍광과, 웅진해안도로와 함덕해수욕장 등 아름'

In [37]:
## Chat Model

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

chat_prompt = ChatPromptTemplate([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("user", "{user_input}")
])

chain = chat_prompt | chat
chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해주세요."})

AIMessage(content='안녕하세요! 한국의 대표적인 관광지 중에서 세 곳을 추천드리겠습니다:\n\n1. 경복궁 (Gyeongbokgung): 서울에 위치한 경복궁은 조선 시대 궁궐 중 가장 크고 아름다운 궁전이며, 한국의 역사와 전통을 체험할 수 있는 곳입니다. 경복궁 주변에는 국립고궁박물관과 창덕궁 등도 함께 방문할 수 있습니다.\n\n2. 부산 해운대해수욕장 (Haeundae Beach): 부산의 대표적인 해변인 해운대해수욕장은 긴 백사장과 선명한 바다가 아름다운 곳으로, 해안가의 다양한 레스토랑과 카페, 그리고 호텔 등 편의시설도 잘 갖추어져 있습니다. 여름에 해운대를 방문하면 해수욕과 해변 산책을 즐길 수 있습니다.\n\n3. 경주 (Gyeongju): 경주는 한국의 역사와 문화가 깃든 도시로, 세계문화유산으로 지정된 석굴암이나 첨성대, 안압지, 불국사 등 다양한 유적지가 위치해 있습니다. 또한, 경주는 자연이 아름다운 곳으로 도립공원이나 황리단길 등을 방문해 휴식을 취할 수도 있습니다.\n\n이 세 군데가 한국 여행을 즐기는 데 도움이 되리라 기대됩니다. 부가적인 정보가 필요하시면 언제든지 물어보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 509, 'prompt_tokens': 59, 'total_tokens': 568, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_

## 1-4-2 LangChain의 LLM 모델 파라미터 설정

- Temperature: 생성된 텍스트의 다양성을 조정. 값이 작으면 예측 가능하고 일관된 출력을 생성하는 반면, 값이 크면 다양하고 예측하기 어려운 출력을 생성.
- Max Tokens (최대 토큰 수): 생성할 최대 토큰 수를 지정. 생성할 텍스트의 길이를 제한.
- Top P (Top Probability): 생성 과정에서 특정 확률 분포 내에서 상위 P% 토큰만을 고려하는 방식. 이는 출력의 다양성을 조정하는 데 도움이 됨.
- Frequency Penalty (빈도 패널티): 값이 클수록 이미 등장한 단어나 구절이 다시 등장할 확률을 감소시킴. 이를 통해 반복을 줄이고 텍스트의 다양성을 증가시킬 수 있음. (0~1)
- Presence Penalty (존재 패널티): 텍스트 내에서 단어의 존재 유무에 따라 그 단어의 선택 확률을 조정. 값이 클수록 아직 텍스트에 등장하지 않은 새로운 단어의 사용이 장려됨. (0~1)
- Stop Sequences (정지 시퀀스): 특정 단어나 구절이 등장할 경우 생성을 멈추도록 설정. 이는 출력을 특정 포인트에서 종료하고자 할 때 사용됨.

In [39]:
from langchain_openai import ChatOpenAI

# 모델 파라미터 설정
params = {
    "temperature": 0.7,     # 생성된 텍스트의 다양성 조정
    "max_tokens": 100,       # 생성된 최대 토큰 수
}

kwargs = {
    "frequency_penalty": 0.5,   # 이미 등장한 단어의 재등장 확률을 조정
    "presence_penalty": 0.5,    # 새로운 단어의 도입을 장려
    "stop": ["\n"]              # 정지 시퀀스 설정
}

# 모델 인스턴스를 생성할 때 설정
model = ChatOpenAI(model= "gpt-3.5-turbo-0125", **params, model_kwargs= kwargs)

# 모델 호출
question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = model.invoke(input = question)

print(response)

c:\Users\mmm06\anaconda3\envs\LangChain\lib\site-packages\IPython\core\interactiveshell.py:3517: UserWarning: Parameters {'presence_penalty', 'stop', 'frequency_penalty'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


content='가장 큰 행성은 목성입니다. 목성은 태양계에서 가장 큰 질량을 가지고 있으며, 지름도 가장 큽니다. 이 행성은 다른 행성들과 비교했을 때 압도적으로 크기 때문에 태양계의 거대한 행성 중 하나로' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 99, 'prompt_tokens': 29, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None} id='run-1b04210a-55ef-4ebb-b201-64766d168e5c-0' usage_metadata={'input_tokens': 29, 'output_tokens': 99, 'total_tokens': 128, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 모델에 직접 파라미터를 전달 하는 방식

In [42]:
# 모델 파라미터 설정
params = {
    "temperature": 0.7,     # 생성된 텍스트의 다양성 조정
    "max_tokens": 10,       # 생성된 최대 토큰 수
}

# 모델 인스턴스를 호출할 떄 전달
response = model.invoke(input = question, **params)

# 답변 출력
print(response)

content='태양계에서 가장 큰' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 29, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'length', 'logprobs': None} id='run-4a568687-3129-44a9-a191-456f804fe010-0' usage_metadata={'input_tokens': 29, 'output_tokens': 10, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 모델에 추가적인 파라미터를 전달

- bind 메서드를 사용하여 체인에 새로운 파라미터를 추가하여 연결 가능
- 다양한 상황에 맞게 모델의 동작을 제어

In [44]:
from langchain_core.prompts import ChatPromptTemplate

#
chat_prompt = ChatPromptTemplate([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

model = ChatOpenAI(model = 'gpt-3.5-turbo-0125', max_completion_tokens= 100)

messages = chat_prompt.format_messages(user_input = "태양계에서 가장 큰 행성은 무엇인가요?")

before_answer = model.invoke(messages)

# binding 이전 출력
print(before_answer)

# 모델 호출 시 추가적인 인수를 전달하기 위해 bind 메서드 사용 (응답의 최대 길이를 10 토큰으로 제한)
chain = chat_prompt | model.bind(max_tokens = 10)

# binding 이후 출력
after_answer = chain.invoke(messages)
print(after_answer)

content='가장 큰 행성은 목성입니다. 목성은 태양계에서 가장 큰 천체로 알려져 있고, 매우 큰 질량을 가지고 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 58, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-928b4eef-62ba-48e3-9d1c-27dcfdc654fe-0' usage_metadata={'input_tokens': 58, 'output_tokens': 57, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='태양계에서 가장 큰' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 106, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, '

## Part 2. RAG(Retrieval-Augumented Generation)

- 기존의 대규모 언어 모델(LLM)을 확장하여, 주어진 컨텍스트나 질문에 대해 더욱 정확하고 풍부한 정보를 제공하는 방법.
- 모델이 학습 데이터에 포함되지 않은 외부 데이터를 실시간으로 검색(Retrieval)하고, 이를 바탕으로 답변을 생성(Generation)하는 과정을 포함
- 특히, 환각(생성된 내용이 사실이 아닌 것으로 오인되는 현상)을 방지하고, 모델이 최신 정보를 반영하거나 더 넓은 지식을 활용할 수 있게 함


1. RAG 모델의 기본 구조
- 검색 단계: 사용자의 질문이나 컨텍스트를 입력으로 받아서, 이와 관련된 외부 데이터를 검색하는 단계
- 생성 단계: 검색된 데이터를 기반으로 LLM 모델이 사용자의 질문에 답변을 생성하는 단계

2. RAG 모델의 장점
- 풍부한 정보 제공: RAG 모델은 검색을 통해 얻은 외부 데이터를 활용하여, 보다 구체적이고 풍부한 정보를 제공
- 실시간 정보 반영: 최신 데이터를 검색하여 반영함으로써, 모델이 실시간으로 변화하는 정보에 대응
- 환각 방지: 검색을 통해 실제 데이터에 기반한 답변을 생성함으로써, 환각 현상이 발생할 위험을 줄이고 정확도를 높임임

In [45]:
import langchain

langchain.__version__

'0.3.15'

## RAG 파이프라인 개요

- Load data - Text Split - Indexing - Retrieval - Generation

In [ ]:
!pip install bs4
!pip install langchain_community
!pip install chromadb

In [71]:
# Data Loader - 웹 페이지 정보 가져오기
import bs4
from langchain_community.document_loaders import WebBaseLoader


url = "https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8"
loader = WebBaseLoader(url)

# 웹페이지 텍스트 -> Documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

1
13283
 위해 좀 더 빠르게 강력한 수단을 이용해야 합니다. 특히 정책 문서에 명시된 원칙을 지키지 않는 것은 대부분의 경우 다른 사용자에게 받아들여지지 않습니다 (다른 분들에게 예외 상황임을 설득할 수 있다면 가능하기는 하지만요). 이는 당신을 포함해서 편집자 개개인이 정책과 지침을 직접 집행 및 적용한다는 것을 의미합니다.
특정 사용자가 명백히 정책에 반하는 행동을 하거나 정책과 상충되는 방식으로 지침을 어기는 경우, 특히 의도적이고 지속적으로 그런 행위를 하는 경우 해당 사용자는 관리자의 제재 조치로 일시적, 혹은 영구적으로 편집이 차단될 수 있습니다. 영어판을 비롯한 타 언어판에서는 일반적인 분쟁 해결 절차로 끝낼 수 없는 사안은 중재위원회가 개입하기도 합니다.

문서 내용
정책과 지침의 문서 내용은 처음 읽는 사용자라도 원칙과 규범을 잘 이해할 수 있도록 다음 원칙을 지켜야 합니다.

명확하게 작성하세요. 소수만 알아듣거나 준법률적인 단어, 혹은 지나치게 단순한 표현은 피해야 합니다. 명확하고, 직접적이고, 모호하지 않고, 구체적으로 작성하세요. 지나치게 상투적인 표현이나 일반론은 피하세요. 지침, 도움말 문서 및 기타 정보문 문서에서도 "해야 합니다" 혹은 "하지 말아야 합니다" 같이 직접적인 표현을 굳이 꺼릴 필요는 없습니다.
가능한 간결하게, 너무 단순하지는 않게. 정책이 중언부언하면 오해를 부릅니다. 불필요한 말은 생략하세요. 직접적이고 간결한 설명이 마구잡이식 예시 나열보다 더 이해하기 쉽습니다. 각주나 관련 문서 링크를 이용하여 더 상세히 설명할 수도 있습니다.
규칙을 만든 의도를 강조하세요. 사용자들이 상식대로 행동하리라 기대하세요. 정책의 의도가 명료하다면, 추가 설명은 필요 없죠. 즉 규칙을 '어떻게' 지키는지와 더불어 '왜' 지켜야 하는지 확실하게 밝혀야 합니다.
범위는 분명히, 중복은 피하기. 되도록 앞부분에서 정책 및 지침의 목적과 범위를 분명하게 밝혀야 합니다. 독자 대부분은 도입부 초반만 읽고 나가버리니까요. 각 정책 

In [72]:
# Text Split (Document -> small chumks: Documents)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)

print(len(splits))
print(splits[10])

19
page_content='제안과 채택
 백:아님 § 관료주의  문서를 참고하십시오. 단축백:제안
제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.
'제안'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.
좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.
정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.' metadata={'source': '

In [73]:
# page 콘텐츠 속성
splits[10].page_content

'제안과 채택\n\xa0백:아님 §\xa0관료주의  문서를 참고하십시오. 단축백:제안\n제안 문서란 정책과 지침으로 채택하자고 의견을 묻는 문서이나 아직 위키백과 내에 받아들여지는 원칙으로 확립되지는 않은 문서입니다. {{제안}} 틀을 붙여 공동체 내에서 정책이나 지침으로 채택할 지 의견을 물을 수 있습니다. 제안 문서는 정책과 지침이 아니므로 아무리 실제 있는 정책이나 지침을 요약하거나 인용해서 다른 문서에 쓴다고 해도 함부로 정책이나 지침 틀을 붙여서는 안 됩니다.\n\'제안\'은 완전 새로운 원칙이라기보다, 기존의 불문율이나 토론 총의의 문서를 통한 구체화에 가깝습니다. 많은 사람들이 쉽게 제안을 받아들이도록 하기 위해서는, 기초적인 원칙을 우선 정하고 기본 틀을 짜야 합니다. 정책과 지침의 기본 원칙은 "왜 지켜야 하는가?", "어떻게 지켜야 하는가?" 두 가지입니다. 특정 원칙을 정책이나 지침으로 확립하기 위해서는 우선 저 두 가지 물음에 성실하게 답하는 제안 문서를 작성해야 합니다.\n좋은 아이디어를 싣기 위해 사랑방이나 관련 위키프로젝트에 도움을 구해 피드백을 요청할 수 있습니다. 이 과정에서 공동체가 어느 정도 받아들일 수 있는 원칙이 구체화됩니다. 많은 이와의 토론을 통해 공감대가 형성되고 제안을 개선할 수 있습니다.\n정책이나 지침은 위키백과 내의 모든 편집자들에게 적용되는 원칙이므로 높은 수준의 총의가 요구됩니다. 제안 문서가 잘 짜여졌고 충분히 논의되었다면, 더 많은 공동체의 편집자와 논의를 하기 위해 승격 제안을 올려야 합니다. 제안 문서 맨 위에 {{제안}}을 붙여 제안 안건임을 알려주고, 토론 문서에 {{의견 요청}}을 붙인 뒤 채택 제안에 관한 토론 문단을 새로 만들면 됩니다. 많은 편집자들에게 알리기 위해 관련 내용을 {{위키백과 소식}}에 올리고 사랑방에 이를 공지해야 하며, 합의가 있을 경우 미디어위키의 sitenotice(위키백과 최상단에 노출되는 구역)에 공지할 수도 있습니다.'

In [74]:
# metadata 속성
splits[10].metadata

{'source': 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8',
 'title': '위키백과:정책과 지침 - 위키백과, 우리 모두의 백과사전',
 'language': 'ko'}

In [80]:
# Indexing (Texts -> Embedding -> Store)
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")

print(len(docs))
print(docs[0].page_content)

4
격하
특정 정책이나 지침이 편집 관행이나 공동체 규범이 바뀌며 쓸모없어질 수 있고, 다른 문서가 개선되어 내용이 중복될 수 있으며, 불필요한 내용이 증식할 수도 있습니다. 이 경우 편집자들은 정책을 지침으로 격하하거나, 정책 또는 지침을 보충 설명, 정보문, 수필 또는 중단 문서로 격하할 것을 제안할 수 있습니다. 
격하 과정은 채택 과정과 비슷합니다. 일반적으로 토론 문서 내 논의가 시작되고 프로젝트 문서 상단에 {{새로운 토론|문단=진행 중인 토론 문단}} 틀을 붙여 공동체의 참여를 요청합니다. 논의가 충분히 이루어진 후, 제3의 편집자가 토론을 종료하고 평가한 후 상태 변경 총의가 형성되었는지 판단해야 합니다. 폐지된 정책이나 지침은 최상단에 {{중단}} 틀을 붙여 더 이상 사용하지 않는 정책/지침임을 알립니다.
소수의 공동체 인원만 지지하는 수필, 정보문 및 기타 비공식 문서는 일반적으로 주된 작성자의 사용자 이름공간으로 이동합니다. 이러한 논의는 일반적으로 해당 문서의 토론란에서 이루어지며, 간혹 위키백과:의견 요청을 통해 처리되기도 합니다.

같이 보기
위키백과:위키백과의 정책과 지침 목록
위키백과:의견 요청
수필

위키백과:제품, 절차, 정책
위키백과:위키백과 공동체의 기대와 규범
기타 링크


In [83]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt

template = '''Answer the question based only on the following context:
{context}

Qeustion: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM(Temperature는 답변의 일관성을 위해 0으로 설정)
model = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature=0)

# Retriever
retreiver = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG chain 연결
rag_chain = (
    {'context': retreiver | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# chain 실행
rag_chain.invoke("격하 과정에 대해 설명해주세요.")

'격하 과정은 특정 정책이나 지침이 더 이상 필요하지 않거나 개선이 필요한 경우에 해당 정책이나 지침을 수정하거나 중단하는 과정을 말합니다. 이를 위해 편집자들은 해당 정책이나 지침을 보충 설명, 정보문, 수필 또는 중단 문서로 변경하거나 폐지할 것을 제안하게 됩니다. 이러한 과정은 토론을 거쳐 결정되며, 폐지된 정책이나 지침은 {{중단}} 틀을 붙여 더 이상 사용되지 않음을 알립니다.'